In [11]:
from pathlib import Path
import os 

import geopandas as gpd


### Read the GeoJSON file and extract the target polygon


In [233]:
test_filename = 'test.geojson'
test_filepath = Path(os.getcwd()).parent / 'data' / test_filename
# Read the GeoJSON file

gdf = gpd.read_file(test_filepath)


In [240]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [238]:
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

def polygon_binning_to_points(gdf: gpd.GeoDataFrame, resolution_m: int) -> gpd.GeoDataFrame:
    '''
    Assume that the target polygon is using WGS 84 (EPSG:4326) and that the 
    GeoDataFrame contains only one polygon
    '''
    target_polygon = gdf.geometry[0]  # Assuming there's only one polygon in the file

    # Create a GeoDataFrame with the target polygon
    gdf_target = gpd.GeoDataFrame(geometry=[target_polygon], crs=gdf.crs)

    # Reproject the GeoDataFrame to the desired CRS (e.g., Web Mercator)
    crs_3857 = "EPSG:3857"
    gdf_target_3857 = gdf_target.to_crs(crs_3857)

    # Determine the bounding box of the projected polygon
    minx, miny, maxx, maxy = gdf_target_3857.geometry.bounds.values[0]

    # Generate grid points within the bounding box
    grid_points = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            point = Point(x, y)
            if gdf_target_3857.geometry[0].contains(point):
                grid_points.append(point)
            y += resolution_m
        x += resolution_m

    grid_geopd = gpd.GeoDataFrame(geometry=grid_points, crs=crs_3857)

    grid_geopd = grid_geopd.to_crs(crs = CRS.from_epsg(4326))

    return grid_geopd



In [239]:
grid_geopd = polygon_binning_to_points(gdf = gdf, resolution_m = 250)
grid_geopd

,geometry
0,POINT (5.33817 43.49912)
1,POINT (5.33817 43.50075)
2,POINT (5.33817 43.50238)
3,POINT (5.34042 43.49750)
4,POINT (5.34042 43.49912)
...,...
159,POINT (5.37186 43.50401)
160,POINT (5.37411 43.49912)
161,POINT (5.37411 43.50075)
162,POINT (5.37411 43.50238)


In [242]:
import requests

def get_all_soilgrid_for_points(points: gpd.GeoSeries):
    '''
    We will use a GeoSeries as an input, each value must be a Shapely Point
    '''
    # URL and endpoint for the API
    url = 'https://isqaper.isric.org/isqaper-rest/api/1.0/query'

    # Create an empty list to store the API responses
    api_responses = []

    # Iterate over each grid point and query the API
    for point in points.geometry:
        lat, lon = point.x, point.y  # Extract the longitude and latitude from the grid point
        params = {'lon': lon, 'lat': lat}  # API parameters for the coordinates

        # Send GET request to the API
        response = requests.get(url, params=params)

        if response.status_code == 200:
            # Successful response
            data = response.json()
            api_responses.append(data)
        else:
            # Handle error response
            print(f"Error: {response.status_code}")

    return api_responses

In [243]:
api_responses = get_all_soilgrid_for_points(points = grid_geopd)

In [244]:
def format_soilgrid_response(api_responses):
    '''
    Format soilgrid api responses
    some type of data such as biological properties have an additional nested structure. 
    The function first browse the structure of the first response and if it finds a metadata
    field, it extracts the values of all responses and store them in the column name after
    the short description field.
    
    NB: some field like Depth to bedrock and Phosphorus using the Olsen method are categorical,
    they will need additional processing to be usable, but in our polygon of interest, all 
    values were identical (>200 and None, respectively)
    '''
    coordinates = [[feat['geometry']['coordinates'] for feat in api_response['features']] for api_response in api_responses]
    # Create a list of Shapely Point geometries from the coordinates
    geometries = [Point(coord) for coord in coordinates]
    # Create the GeoDataFrame with points coordinates only
    gdf = gpd.GeoDataFrame(geometry=geometries)

    # browse the nested structure of api response dictionary and retrieve short description and values
    # and then fill the geodataframe column by column for all responses
    for feat_key , feat_value in api_responses[0]['features'][0]['properties']['properties'].items():
        if 'metadata' in feat_value.keys():
            gdf[feat_value['metadata']['short_description']] = [api_response['features'][0]['properties']['properties'][feat_key]['value'] for api_response in api_responses]

        else:
            for nested_key , nested_value in feat_value.items():
                gdf[nested_value['metadata']['short_description']] = [api_response['features'][0]['properties']['properties'][feat_key][nested_key]['value'] for api_response in api_responses]

    return gdf

soilgrid_gdf = format_soilgrid_response(api_responses)
soilgrid_gdf

,geometry,Soil water capacity (pF=2.0),Soil water capacity (pF=2.3),Soil water capacity (pF=2.5),Permanent wilting point (PWP),Depth to bedrock,Bulk density (fine earth),Cation exchange capacity,Clay,Coarse fragments (volume),...,Soil pH,Plant-available water storage capacity,Silt,Sand,Electrical conductivity,Exchangeable potassium,Phosphorus using the Olsen method,Total nitrogen in soil,Estimated soil microbial abundance,Macrofauna groups
0,POINT (43.49912 5.33817),14.0,10.0,9.0,20.0,>200,1472.0,24.0,23.0,18.0,...,7.9,42.0,23.0,53.0,0.3,0.54,None,0.84,14.0,3.0
1,POINT (43.50075 5.33817),14.0,11.0,9.0,20.0,>200,1462.0,24.0,22.0,21.0,...,7.9,42.0,24.0,53.0,0.3,0.54,None,0.84,14.0,3.0
2,POINT (43.50238 5.33817),15.0,11.0,9.0,20.0,>200,1449.0,23.0,22.0,27.0,...,7.9,45.0,24.0,53.0,0.3,0.54,None,0.84,14.0,3.0
3,POINT (43.49750 5.34042),14.0,10.0,8.0,20.0,>200,1472.0,24.0,22.0,18.0,...,8.0,42.0,23.0,53.0,0.3,0.54,None,0.84,14.0,3.0
4,POINT (43.49912 5.34042),14.0,11.0,9.0,20.0,>200,1469.0,24.0,22.0,18.0,...,8.0,42.0,24.0,53.0,0.3,0.54,None,0.84,14.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,POINT (43.50401 5.37186),14.0,11.0,9.0,19.0,>200,1482.0,21.0,24.0,28.0,...,7.9,42.0,24.0,51.0,0.3,0.54,None,0.84,14.0,3.0
160,POINT (43.49912 5.37411),14.0,10.0,8.0,19.0,>200,1495.0,22.0,23.0,25.0,...,7.9,42.0,24.0,52.0,0.3,0.54,None,0.84,14.0,3.0
161,POINT (43.50075 5.37411),13.0,10.0,8.0,20.0,>200,1494.0,22.0,23.0,27.0,...,7.9,39.0,23.0,52.0,0.3,0.54,None,0.84,14.0,3.0
162,POINT (43.50238 5.37411),14.0,11.0,9.0,19.0,>200,1485.0,21.0,23.0,27.0,...,7.9,42.0,24.0,52.0,0.3,0.54,None,0.84,14.0,3.0


In [245]:
soilgrid_gdf['Depth to bedrock'].unique()

array(['>200'], dtype=object)